In [ ]:
import os
import time
import math
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm
from itertools import combinations

# 1) Load the full population dataset
x_population = pd.read_csv("data/h_population.csv")

# 2) Encode each categorical column as integer labels
label_encoders = {}
for col in x_population.columns:
    le = LabelEncoder()
    x_population[col] = le.fit_transform(x_population[col])
    label_encoders[col] = le

# 3) Convert columns to pandas 'category' dtype (expected by OneHotEncoder)
x_population = x_population.apply(lambda col: col.astype("category"))

# 4) Create and fit the OneHotEncoder
#    Note: use sparse_output=False in scikit-learn >=1.2
enc = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
enc.fit(x_population)

# 5) Load your actual sample data
# # Example of Spliting 5% Training data
# x_train_split = train_test_split(
#     x_population, test_size=0.05, shuffle=True, random_state=1004
# )[1]
x_train_split = pd.read_csv("data/h_sample.csv")

# 6) Apply the same LabelEncoder mappings to the sample data
for col in x_train_split.columns:
    x_train_split[col] = label_encoders[col].transform(x_train_split[col])

# 7) (Optional) Convert sample data to 'category' dtype as well
x_train_split = x_train_split.apply(lambda col: col.astype("category"))

# 8) Perform one-hot encoding on the sample
x_train_encoded = enc.transform(x_train_split)

# 9) Convert the encoded array into a PyTorch tensor
x_train_tensor = torch.tensor(x_train_encoded, dtype=torch.float32)

print("Training data shape:", x_train_tensor.shape)


Training data shape: torch.Size([53315, 70])


In [2]:
# 2. MODEL DEFINITION
class VanillaVAE(nn.Module):
    def __init__(self, input_dim, hidden_dims, latent_dim, cat_dims):
        super().__init__()
        # Encoder
        enc_layers = []
        prev = input_dim
        for h in hidden_dims:
            enc_layers += [nn.Linear(prev, h), nn.ReLU()]
            prev = h
        self.encoder = nn.Sequential(*enc_layers)
        self.fc_mu     = nn.Linear(hidden_dims[-1], latent_dim)
        self.fc_logvar = nn.Linear(hidden_dims[-1], latent_dim)
        # Decoder
        dec_layers = []
        prev = latent_dim
        for h in reversed(hidden_dims):
            dec_layers += [nn.Linear(prev, h), nn.ReLU()]
            prev = h
        self.decoder_hidden = nn.Sequential(*dec_layers)
        # Output heads
        self.dec_heads = nn.ModuleList([
            nn.Linear(hidden_dims[0], cd) for cd in cat_dims
        ])

    def encode(self, x):
        h = self.encoder(x)
        return self.fc_mu(h), self.fc_logvar(h)

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def decode(self, z):
        h = self.decoder_hidden(z)
        out = [head(h) for head in self.dec_heads]
        return out

    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)
        logits = self.decode(z)
        return logits, mu, logvar

In [3]:
##########################
# 3. Utils
##########################

def wide_to_long(samples_pop, cat_boundaries, enc):
    """
    Converts the wide one-hot output to discrete categories
    by sampling from each column's predicted probabilities.
    """
    resamples = []
    for row_idx in range(samples_pop.shape[0]):
        row_probs = samples_pop[row_idx]
        resample_row = []
        for i in range(len(cat_boundaries) - 1):
            start = cat_boundaries[i]
            end   = cat_boundaries[i+1]
            probs = row_probs[start:end]
            sum_p = probs.sum()
            if sum_p > 0:
                probs /= sum_p
            else:
                probs = np.ones_like(probs) / len(probs)

            chosen_idx = np.random.choice(len(probs), p=probs)
            original_value = enc.categories_[i][chosen_idx]
            resample_row.append(original_value)
        resamples.append(resample_row)

    df = pd.DataFrame(resamples, columns=enc.feature_names_in_)
    return df.apply(lambda x: x.astype('category'))

def SRMSE(x_population: pd.DataFrame, resamples: pd.DataFrame):
    """Caculate Marginal·Bivariate SRMSE"""
    sam_marg_cnt, resam_marg_cnt = [], []
    for col in x_population.columns:
        pop_series = x_population[col].dropna()
        syn_series = resamples[col].dropna()

        tab = (pd
               .merge(syn_series.value_counts().rename('syn'),
                      pop_series.value_counts().rename('pop'),
                      left_index=True, right_index=True, how='outer')
               .fillna(0))
        sam_marg_cnt.append(tab['pop'].values / (len(pop_series) or 1))
        resam_marg_cnt.append(tab['syn'].values / (len(syn_series) or 1))

    sam_marg_cnt = np.concatenate(sam_marg_cnt) if sam_marg_cnt else np.array([])
    resam_marg_cnt = np.concatenate(resam_marg_cnt) if resam_marg_cnt else np.array([])

    if sam_marg_cnt.size:
        rmse_mar  = np.linalg.norm(sam_marg_cnt - resam_marg_cnt) / np.sqrt(len(sam_marg_cnt))
        ybar_mar  = sam_marg_cnt.mean()
        srmse_mar = rmse_mar / ybar_mar if ybar_mar else np.nan
    else:
        srmse_mar = np.nan

    # ── Bivariate ──────────────────────────────────────────
    sam_bi_cnt, resam_bi_cnt = [], []
    for c1, c2 in combinations(x_population.columns, 2):
        pop_pair = x_population[[c1, c2]].dropna()
        syn_pair = resamples[[c1, c2]].dropna()
        if pop_pair.empty or syn_pair.empty:
            continue  

        tab = (pd
               .merge(pd.crosstab(syn_pair[c1], syn_pair[c2]).stack().rename('syn'),
                      pd.crosstab(pop_pair[c1], pop_pair[c2]).stack().rename('pop'),
                      left_index=True, right_index=True, how='outer')
               .fillna(0))
        sam_bi_cnt.append(tab['pop'].values / (len(pop_pair) or 1))
        resam_bi_cnt.append(tab['syn'].values / (len(syn_pair) or 1))

    sam_bi_cnt  = np.concatenate(sam_bi_cnt)  if sam_bi_cnt  else np.array([])
    resam_bi_cnt = np.concatenate(resam_bi_cnt) if resam_bi_cnt else np.array([])

    if sam_bi_cnt.size:
        rmse_bi  = np.linalg.norm(sam_bi_cnt - resam_bi_cnt) / np.sqrt(len(sam_bi_cnt))
        ybar_bi  = sam_bi_cnt.mean()
        srmse_bi = rmse_bi / ybar_bi if ybar_bi else np.nan
    else:
        srmse_bi = np.nan

    return srmse_mar, srmse_bi


def calculate_precision_recall(population_df: pd.DataFrame,
                               generated_df: pd.DataFrame):
    """Precision·Recall·F1 and combinations"""
    pop_df = population_df.dropna()
    gen_df = generated_df.dropna()
    if pop_df.empty or gen_df.empty:
        return dict(precision=0, recall=0, f1_score=0,
                    unique_combinations={'population': 0, 'generated': 0},
                    matching_combinations={'unique_types': 0, 'total_count': 0})

    pop_tuples = pop_df.apply(tuple, axis=1)
    gen_tuples = gen_df.apply(tuple, axis=1)

    pop_set, gen_set = set(pop_tuples), set(gen_tuples)

    precision = (gen_tuples.isin(pop_set)).mean()
    recall    = (pop_tuples.isin(gen_set)).mean()
    f1        = 2*precision*recall/(precision+recall) if precision+recall else 0

    matching_unique = pop_set & gen_set
    gen_counts      = gen_tuples.value_counts()
    match_row_cnt   = sum(gen_counts[t] for t in matching_unique)

    return {
        'precision' : round(float(precision), 4),
        'recall'    : round(float(recall),    4),
        'f1_score'  : round(float(f1),        4),
        'unique_combinations': {
            'population': int(pop_tuples.nunique()),
            'generated' : int(gen_tuples.nunique())
        },
        'matching_combinations': {
            'unique_types': len(matching_unique),
            'total_count' : int(match_row_cnt)
        }
    }

def evaluate_coverage(
    x_population: pd.DataFrame,
    x_sample: pd.DataFrame,
    generator,
    cat_boundaries,
    enc,
    device,
    latent_dim: int,
    batch_size: int = 256,
):
    """
    1) Generate the same number of synthetic samples as the population
    2) Compute STZ/SAZ ratios, SRMSE, and Precision/Recall/F1
    3) Return a dictionary containing all metrics
    """
    # ── 1. Synthetic data generation ───────────────────────────────────────
    generator.eval().to(device)
    n_gen = len(x_population)
    results, done = [], 0
    t0 = time.time()

    with tqdm(total=n_gen, desc="Generating") as bar:
        while done < n_gen:
            cur = min(batch_size, n_gen - done)
            noise = torch.randn(cur, latent_dim, device=device)
            with torch.no_grad():
                fake = generator(noise).cpu().numpy()
            df_cat = wide_to_long(fake, cat_boundaries, enc)
            results.append(df_cat)
            done += cur
            bar.update(cur)

    resamples = pd.concat(results, ignore_index=True)
    infer_sec = time.time() - t0

    # ── 2. Common preprocessing (string conversion & column alignment) ─────
    pop_df = x_population.astype(str).reset_index(drop=True)
    sam_df = x_sample.astype(str).reset_index(drop=True)
    gen_df = resamples.astype(str).reindex(columns=pop_df.columns)

    # ── 3-A. STZ / SAZ computation ─────────────────────────────────────────
    pop_key = pop_df.agg("".join, axis=1)
    sam_key = sam_df.agg("".join, axis=1)
    gen_key = gen_df.agg("".join, axis=1)

    STZ = gen_df[~gen_key.isin(sam_key) & ~gen_key.isin(pop_key)]
    SAZ = gen_df[~gen_key.isin(sam_key) & gen_key.isin(pop_key)]
    stz_ratio = round(len(STZ) / len(gen_df), 4)
    saz_ratio = round(len(SAZ) / len(gen_df), 4)

    # ── 3-B. SRMSE computation ─────────────────────────────────────────────
    srmse_mar, srmse_bi = SRMSE(pop_df, gen_df)

    # ── 3-C. Precision, Recall, F1 computation ─────────────────────────────
    pr = calculate_precision_recall(pop_df, gen_df)

    # ── 4. Packaging results ───────────────────────────────────────────────
    metrics = {
        "stz_ratio": stz_ratio,
        "saz_ratio": saz_ratio,
        "srmse_marginal": srmse_mar,
        "srmse_bivariate": srmse_bi,
        "precision": pr["precision"],
        "recall": pr["recall"],
        "f1_score": pr["f1_score"],
        "unique_combinations": pr["unique_combinations"],
        "matching_combinations": pr["matching_combinations"],
        "infer_time_sec": round(infer_sec, 2),
    }
    return metrics

In [4]:
device           = torch.device("cuda" if torch.cuda.is_available() else "cpu")
cat_dims         = [x_population[col].nunique() for col in x_population.columns]
total_output_dim = sum(cat_dims)
cat_boundaries   = [0] + list(np.cumsum(cat_dims))

input_dim = sum(cat_dims)
latent_dim  = 32
hidden_dims = [256, 128, 64]
batch_size  = 256
epochs      = 250

train_dataset = TensorDataset(x_train_tensor)
loader        = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

vae       = VanillaVAE(input_dim, hidden_dims, latent_dim, cat_dims).to(device)
optimizer = optim.Adam(vae.parameters(), lr=0.0005, betas=(0.5, 0.9))

os.makedirs("Vanilla-VAE", exist_ok=True)
best_vae_loss = float("inf")
save_interval = 50
train_start   = time.time()

for epoch in tqdm(range(1, epochs+1), desc="Training Progress"):
    vae.train()
    epoch_loss   = 0.0
    batch_count  = 0

    total_recon  = 0.0
    total_kl     = 0.0

    for (xb,) in tqdm(loader, desc=f"Epoch {epoch}/{epochs}", leave=False):
        xb = xb.to(device)
        batch_count += 1

        optimizer.zero_grad()
        logits_list, mu, logvar = vae(xb)

        # 1) Reconstruction loss
        recon_loss = 0.0
        for i, logits in enumerate(logits_list):
            start, end   = cat_boundaries[i], cat_boundaries[i+1]
            true_onehot  = xb[:, start:end]
            target_idx   = torch.argmax(true_onehot, dim=1)
            recon_loss  += F.cross_entropy(logits, target_idx)

        # 2) KL divergence
        kl_per_sample = -0.5 * torch.sum(
            1 + logvar - mu.pow(2) - logvar.exp(),
            dim=1
        )
        kl = torch.mean(kl_per_sample)

        # 3) ELBO loss
        loss = recon_loss + kl
        loss.backward()
        optimizer.step()

        epoch_loss  += loss.item()
        total_recon += recon_loss.item()
        total_kl    += kl.item()

    avg_loss  = epoch_loss  / batch_count
    avg_recon = total_recon / batch_count
    avg_kl    = total_kl    / batch_count

    if epoch % 10 == 0:
        tqdm.write(
            f"[Epoch {epoch}/{epochs}]  "
            f"Recon: {avg_recon:.3f}  KL: {avg_kl:.3f}  Total: {avg_loss:.3f}"
        )

    # best/ckpt 저장 로직 그대로
    if avg_loss < best_vae_loss:
        best_vae_loss = avg_loss
        torch.save(vae.state_dict(), "Vanilla-VAE/vae_best.pth")
        tqdm.write(f"  [Epoch {epoch}] Best VAE saved (loss = {avg_loss:.4f})")

    if epoch % save_interval == 0:
        torch.save(vae.state_dict(), f"Vanilla-VAE/vae_epoch_{epoch}.pth")
        tqdm.write(f"  [Epoch {epoch}] Checkpoint saved.")

train_end = time.time()
print(f"\nTraining complete in {train_end - train_start:.2f} seconds")


Training Progress:   0%|▎                                                              | 1/250 [00:02<10:10,  2.45s/it]

  [Epoch 1] Best VAE saved (loss = 15.7398)



Training Progress:   1%|▌                                                              | 2/250 [00:04<09:29,  2.30s/it]

  [Epoch 2] Best VAE saved (loss = 14.5426)



Training Progress:   1%|▊                                                              | 3/250 [00:07<10:35,  2.57s/it]

...

[Epoch 240/250]  Recon: 8.270  KL: 3.477  Total: 11.747



Training Progress:  99%|████████████████████████████████████████████████████████████▌| 248/250 [11:12<00:05,  2.78s/it]

  [Epoch 248] Best VAE saved (loss = 11.7333)



Training Progress: 100%|█████████████████████████████████████████████████████████████| 250/250 [11:17<00:00,  2.71s/it]

[Epoch 250/250]  Recon: 8.262  KL: 3.488  Total: 11.750
  [Epoch 250] Checkpoint saved.

Training complete in 677.86 seconds


In [5]:
device       = "cuda" if torch.cuda.is_available() else "cpu"
latent_dim  = 32
hidden_dims = [256, 128, 64]
cat_dims     = [x_population[col].nunique() for col in x_population.columns]
cat_boundaries = [0] + list(np.cumsum(cat_dims))

input_dim = sum(cat_dims)
vae = VanillaVAE(input_dim, hidden_dims, latent_dim, cat_dims).to(device)
vae.load_state_dict(torch.load("Vanilla-VAE/vae_best.pth", map_location=device))
vae.eval()

class VAESampler(torch.nn.Module):
    def __init__(self, vae, device):
        super().__init__()
        self.vae    = vae
        self.device = device

    def forward(self, noise: torch.Tensor) -> torch.Tensor:
        """
        Input:
          noise: (B, latent_dim) Tensor, 표준 정규분포 샘플
        Returns:
          probs_cat: (B, total_output_dim) Tensor of softmax probabilities
        """
        noise = noise.to(self.device)
        self.vae.eval()
        with torch.no_grad():
            logits_list = self.vae.decode(noise)

        probs = [F.softmax(logits, dim=1) for logits in logits_list]
        probs_cat = torch.cat(probs, dim=1)
        return probs_cat

vae_sampler = VAESampler(vae,device)

evaluate_coverage(
    x_population   = x_population,
    x_sample       = x_train_split,
    generator      = vae_sampler,
    cat_boundaries = cat_boundaries,
    enc            = enc,
    device         = device,
    latent_dim     = latent_dim,
    batch_size     = 256
)


num_samples = x_population.shape[0]
generated_batches = []
batch_size = 256

with torch.no_grad():
    for i in range(0, num_samples, batch_size):
        B = min(batch_size, num_samples - i)
        noise = torch.randn(B, latent_dim, device=device)
        probs_tensor = vae_sampler(noise)           
        probs = probs_tensor.cpu().numpy()           

        df = wide_to_long(probs, cat_boundaries, enc)
        generated_batches.append(df)


generated_data = pd.concat(generated_batches, ignore_index=True)
print("Generated synthetic data shape:", generated_data.shape)
generated_data.to_csv("generated_synthetic_data_VAE.csv", index=False)


Generating: 100%|██████████████████████████████████████████████████████████| 1066319/1066319 [06:48<00:00, 2608.87it/s]


Generated synthetic data shape: (1066319, 13)


In [ ]:
decoded = generated_data.copy()

for col in decoded.columns:
    le = label_encoders[col]
    decoded[col] = le.inverse_transform( decoded[col].astype(int).values )

print(decoded.head())


       Age  Gender  Homeincome  Hometype  CarOwn  Driver  Workdays  Worktype  \
0  [40,45)       1           3         3       1       1         1         6   
1  [35,40)       1           3         1       1       1         3         8   
2  [55,60)       2           3         1       1       1         4         4   
3  [20,25)       2           4         2       1       1         1         5   
4  [45,50)       2           4         3       1       1         1         9   

   Student  NumHH  KidinHH                ComMode     ComTime  
0        4      5        2                    Car        Peak  
1        4      3        1                    Car        Peak  
2        4      2        2             No commute  No commute  
3        4      5        2  Public Transportation        Peak  
4        4      4        2                    Car        Peak  


In [ ]:
decoded.to_csv("generated_synthetic_data_VAE.csv", index=False)